## run kraken2 with CAMI2 AllReads
83029581 sequences (24991.77 Mbp) processed in 2879.342s (1730.2 Kseq/m, 520.78 Mbp/m).

  49536957 sequences classified (59.66%)
  
  33492624 sequences unclassified (40.34%)

In [1]:
import pandas as pd
import numpy as np
import os
import sys
from collections import Counter

In [3]:
def Pre_Sen(lst_true,lst_pre): 
    numOfAmbiguousTrue=0;  ##代表在某个level上，真实值的taxID没找到，赋值为-1
    numOfAmbiguousPredictionTrue=0  ##在某个level上，真实值和预测值的taxID都没找到，赋值为-1
    numOfUnClassified=0   ##没有分类的数量
    numOfCorrectAssign=0;
    numOfAssign=0;
    numOfAll=len(lst_true);
    for i in range(len(lst_true)):
        if(lst_true[i]==-1):
            numOfAmbiguousTrue=numOfAmbiguousTrue+1
        if(lst_true[i]==-1 and lst_pre[i]==-1):##代表没有答案
            numOfAmbiguousPredictionTrue=numOfAmbiguousPredictionTrue+1
            continue
        if(lst_pre[i]==0):
            numOfUnClassified=numOfUnClassified+1
        else:
            numOfAssign=numOfAssign+1;
            if(lst_pre[i]==lst_true[i]):
                numOfCorrectAssign=numOfCorrectAssign+1
    print("numOfAmbiguousTrue: "+str(numOfAmbiguousTrue))
    print("numOfAmbiguousPredictionTrue: "+str(numOfAmbiguousPredictionTrue))
    print("numOfUnClassified: "+str(numOfUnClassified))
    print("numOfCorrectAssign: "+str(numOfCorrectAssign))
    print("numOfAssign: "+str(numOfAssign))
    print("numOfAll: "+str(numOfAll))
    Precision=numOfCorrectAssign/numOfAssign
    Sensitivity=numOfCorrectAssign/numOfAll
    print("Precision: "+str(Precision))
    print("Sensitivity: "+str(Sensitivity))
    F1= 2*Precision*Sensitivity/(Precision+Sensitivity)
    print("F1 Measure: "+str(F1))

In [4]:
### 1：将数据集里分类正确的过滤出来。
def compute_MAPE(TrueLevelcounter,Predict):
    ##计算MAPE
    n=len(TrueLevelcounter)
    result=0.0;
    for k,v in TrueLevelcounter.items():
        R_true=v
        R_est=Predict[k]
        result=result+abs(R_true-R_est)/R_true
        #print("R_true:"+str(R_true)+"   R_est:"+str(R_est)+" Result:"+str(result))
    return result/n*100

def Compute_Abundance(TrueLevel,TargetLevel):
    ##统计答案的species的数量
    TrueLevelcounter=dict(Counter(TrueLevel))
    if( -1 in TrueLevelcounter):
        del TrueLevelcounter[-1]
    if( 0 in TrueLevelcounter):
        del TrueLevelcounter[0]
    keys_true=set(TrueLevelcounter.keys())
    #print(TrueLevelcounter)
    
    ##Target
    TargetLevelcounter=dict(Counter(TargetLevel))
    #print(TargetLevelcounter)
  
    for key in keys_true:
        if key not in TargetLevelcounter:
            TargetLevelcounter[key]=0
    ##计算
    MAPE=compute_MAPE(TrueLevelcounter,TargetLevelcounter)
    print("MAPE: "+str(MAPE))

In [5]:
def compute_Species_ACC(path):
    df=pd.read_csv(path)
    df.columns=['clusterid','readname','clark_clu_species','kra_clu_species','true_taxid','kra_id','clark_taxid','true_species','kra_species','clark_species']
    new_df5=df.set_index('readname')
    print(new_df5.head())
    print(new_df5.shape)
#     lst_true=list(new_df5['true_species'])
#     lst_kra=list(new_df5['kra_species'])
#     lst_clark=list(new_df5['clark_species'])
#     lst_kra_clu=list(new_df5['kra_clu_species'])
#     lst_clark_clu=list(new_df5['clark_clu_species'])
#     print("============================Kraken2 Speices Level Result Show==================================")
#     Pre_Sen(lst_true,lst_kra)
#     Compute_Abundance(lst_true,lst_kra)
#     print("=======================Kraken2 LocalClustering Species Level Result Show============================ ")
#     Pre_Sen(lst_true,lst_kra_clu)
#     Compute_Abundance(lst_true,lst_kra_clu)
#     print("============================Clark Speices Level Result Show==================================")
#     Pre_Sen(lst_true,lst_clark)
#     Compute_Abundance(lst_true,lst_clark)
#     print("=======================Clark LocalClustering Species Level Result Show============================ ")
#     Pre_Sen(lst_true,lst_clark_clu)
#     Compute_Abundance(lst_true,lst_clark_clu)
    return new_df5

In [6]:
def compute_Genus_ACC(path):
    df=pd.read_csv(path)
    df.columns=['clusterid','readname','clark_clu_genus','kra_clu_genus','true_taxid','kra_id','clark_taxid','true_genus','kra_genus','clark_genus']
    new_df4=df.set_index('readname')
    print(new_df4.head())
    print(new_df4.shape)
    lst_true=list(new_df4['true_genus'])
    lst_kra=list(new_df4['kra_genus'])
    lst_clark=list(new_df4['clark_genus'])
    lst_kra_clu=list(new_df4['kra_clu_genus'])
    lst_clark_clu=list(new_df4['clark_clu_genus'])
    print("============================Kraken2 Genus Level Result Show==================================")
    Pre_Sen(lst_true,lst_kra)
    Compute_Abundance(lst_true,lst_kra)
    print("=======================Kraken2 LocalClustering Genus Level Result Show============================ ")
    Pre_Sen(lst_true,lst_kra_clu)
    Compute_Abundance(lst_true,lst_kra_clu)
    print("============================Clark Genus Level Result Show==================================")
    Pre_Sen(lst_true,lst_clark)
    Compute_Abundance(lst_true,lst_clark)
    print("=======================Clark LocalClustering Genus Level Result Show============================ ")
    Pre_Sen(lst_true,lst_clark_clu)
    Compute_Abundance(lst_true,lst_clark_clu)
    return new_df4

In [7]:
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (8, 5)
from matplotlib_venn import venn3
def AnalysisSpecies(df,level):
    ##统计species的数量
    TrueSpcies=set(list(df["true_"+level]))
    CluSpecies=set(list(df["clu_"+level]))
    LstKraSpecies=list(df["kra_"+level])
    DictKraSpecies=dict(Counter(LstKraSpecies))
    KraSpecies=set(DictKraSpecies.keys())
#     KraSpecies=set()
#     for key,val in DictKraSpecies.items():
#         if(val>10):
#             KraSpecies.add(key)
    ##把kraken2的结果，取taxid 5条以上以上的分析
    numOfTrueSpecies=len(TrueSpcies) #数据集里真实species数量
    numOfCluSpecies=len(CluSpecies)  #数量
    numOfKraSpecies=len(KraSpecies)  #数量
    print("numOfTrueSpecies: "+str(numOfTrueSpecies))
    print("numOfCluSpecies: "+str(numOfCluSpecies))
    print("numOfKraSpecies: "+str(numOfKraSpecies))
    ##画venn图
    venn3(subsets=[TrueSpcies,CluSpecies,KraSpecies],set_labels=('True_'+level,'Cluster_'+level,'Kraken_'+level),set_colors=('r','b','g'))
    plt.show()

## Compute the Result of Classification

In [8]:
compute_Species_ACC("allReads_dir/CAMI2_mouse_allreads_tax_species_V3_0.5")

              clusterid  clark_clu_species  kra_clu_species  true_taxid  \
readname                                                                  
S51R10572417     362092             1736.0             1736        1736   
S19R2046636      362092             1736.0             1736        1736   
S33R4008725      362092             1736.0             1736        1736   
S27R10837596    1106303                0.0                0     1432052   
S37R15596687    1106303                0.0                0     1432052   

              kra_id  clark_taxid  true_species  kra_species  clark_species  
readname                                                                     
S51R10572417    1736       1736.0          1736         1736         1736.0  
S19R2046636     1736       1736.0          1736         1736         1736.0  
S33R4008725     1736       1736.0          1736         1736         1736.0  
S27R10837596       0          0.0       1432052            0            0.0  
S37R15

,clusterid,clark_clu_species,kra_clu_species,true_taxid,kra_id,clark_taxid,true_species,kra_species,clark_species
readname,,,,,,,,,
S51R10572417,362092,1736.0,1736,1736,1736,1736.0,1736,1736,1736.0
S19R2046636,362092,1736.0,1736,1736,1736,1736.0,1736,1736,1736.0
S33R4008725,362092,1736.0,1736,1736,1736,1736.0,1736,1736,1736.0
S27R10837596,1106303,0.0,0,1432052,0,0.0,1432052,0,0.0
S37R15596687,1106303,0.0,0,1432052,0,0.0,1432052,0,0.0
...,...,...,...,...,...,...,...,...,...
S58R1117500,934463,0.0,0,1227272,0,0.0,712437,0,0.0
S39R7726363,934463,0.0,0,1227272,0,0.0,712437,0,0.0
S61R2747006,934463,0.0,0,1227272,0,0.0,712437,0,0.0


In [15]:
compute_Genus_ACC("allReads_dir/CAMI2_mouse_allreads_tax_genus_V3_0.5")

              clusterid  clark_clu_genus  kra_clu_genus  true_taxid  kra_id  \
readname                                                                      
S28R7325883     1462634                0              0      575611       0   
S43R12756304    1462634                0              0      575611       0   
S44R16000889    1462634                0              0      575611       0   
S58R15256910    1462634                0              0      575611       0   
S60R6662176     1462634                0              0      575611       0   

              clark_taxid  true_genus  kra_genus  clark_genus  
readname                                                       
S28R7325883           0.0         838          0            0  
S43R12756304          0.0         838          0            0  
S44R16000889          0.0         838          0            0  
S58R15256910          0.0         838          0            0  
S60R6662176           0.0         838          0            0 

,clusterid,clark_clu_genus,kra_clu_genus,true_taxid,kra_id,clark_taxid,true_genus,kra_genus,clark_genus
readname,,,,,,,,,
S28R7325883,1462634,0,0,575611,0,0.0,838,0,0
S43R12756304,1462634,0,0,575611,0,0.0,838,0,0
S44R16000889,1462634,0,0,575611,0,0.0,838,0,0
S58R15256910,1462634,0,0,575611,0,0.0,838,0,0
S60R6662176,1462634,0,0,575611,0,0.0,838,0,0
...,...,...,...,...,...,...,...,...,...
S22R14172369,1855334,0,0,702438,0,0.0,838,0,0
S6R8029807,1855334,0,0,702438,0,0.0,838,0,0
S33R11476065,1976992,0,0,142588,0,0.0,142587,0,0


## Analyze purity and completeness

In [10]:
from matplotlib import pyplot as plt
%pylab inline
import seaborn as sns
sns.set()
plt.rcParams['figure.figsize'] = (12, 8)
%config InlineBackend.figure_format='retina'
import numpy as np
import pandas as pd
import os
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import numpy as numpy
import pandas as pd 
from imp import reload
from sklearn import  metrics

import sys
if not "/home/ubuntu/" in sys.path:
    sys.path.append("/home/ubuntu/")
if not 'function' in sys.modules:
    function = __import__('function')
else:
    eval('import function')
    function = eval('reload(function)')

Populating the interactive namespace from numpy and matplotlib


In [12]:
dfLocal=new_species_df[['readname','clusterid']]
dfLocal.columns=['node','cluster']
ret,dfLocal=function.analysize(label,dfLocal)
print(dfLocal.head())
print(dfLocal.shape)

{'Total #reads': 82279710, 'Total #clusters': 2083160, 'ground #truth': 549, 'Filtered #clusters': 2083160, 'Filtered #reads': 82279710, 'Filtered #ground_truth': 549, 'mean cluster size:': 39.49754699591006, 'median cluster size:': 8.0, '%100 purification:': 0.741892043378314, '>%95 purification:': 0.8222021962855608, '>%90 purification:': 0.8580945628611402, 'mean purification:': 0.9515811060431532, 'median purification:': 1.0, '>%80 purification:': 0.9060694602363029, '>%70 purification:': 0.93254737193551, '>%60 purification:': 0.9624960146283159, '>%50 purification:': 0.9881256548262912, '>%40 purification:': 0.993765592690889, '>%30 purification:': 0.9980030679476148, '>%20 purification:': 0.9999048848888779, '>%10 purification:': 0.9999990294376416, '>%5  purification:': 1.0, '%100 completeness:': 0.0018214936247723133, '>%90 completeness:': 0.0018214936247723133, '>%80 completeness:': 0.0018214936247723133, 'mean completeness:': 0.009235351851917054, 'median completeness:': 0.0

In [13]:
dfGlobal=new_species_df[['readname','kra_clu_species']]
dfGlobal.columns=['node','cluster']
ret,dfGlobal=function.analysize(label,dfGlobal)
print(dfGlobal.head())
print(dfGlobal.shape)

{'Total #reads': 82279710, 'Total #clusters': 7289, 'ground #truth': 549, 'Filtered #clusters': 7289, 'Filtered #reads': 82279710, 'Filtered #ground_truth': 549, 'mean cluster size:': 11288.202771299219, 'median cluster size:': 263.0, '%100 purification:': 0.08166346417787537, '>%95 purification:': 0.10293713972001098, '>%90 purification:': 0.1254460609387867, 'mean purification:': 0.5258508722452367, 'median purification:': 0.47418236757909704, '>%80 purification:': 0.1777381279165523, '>%70 purification:': 0.24979412572055998, '>%60 purification:': 0.34381004666483667, '>%50 purification:': 0.46692286576996983, '>%40 purification:': 0.6174855888004392, '>%30 purification:': 0.7976942080702718, '>%20 purification:': 0.9472961844633544, '>%10 purification:': 0.9987647543233599, '>%5  purification:': 1.0, '%100 completeness:': 0.0036429872495446266, '>%90 completeness:': 0.08196721311475409, '>%80 completeness:': 0.2185792349726776, 'mean completeness:': 0.6894545840021847, 'median comp

In [22]:
df_local_purity=dfLocal.groupby(['cid'])[['c_count','purification']].max().reset_index().sort_values('c_count',ascending=False)
df_local_completeness=dfLocal.groupby(['rname'])[['r_count','completeness']].max().reset_index().sort_values('r_count',ascending=False)
df_global_purity=dfGlobal.groupby(['cid'])[['c_count','purification']].max().reset_index().sort_values('c_count',ascending=False)
df_global_completeness=dfGlobal.groupby(['rname'])[['r_count','completeness']].max().reset_index().sort_values('r_count',ascending=False)
merge=[df_local_completeness['completeness']*100,df_global_completeness['completeness']*100,\
      df_local_purity['purification']*100,df_global_purity['purification']*100]

In [43]:
def percent(values):
    fake=list(sort(values))
    SUM=float(reduce(lambda x,y:x+y,fake))
    j=0
    thd=np.power(10,np.linspace(1,10,60))
   # thd=[100,1000,10000,100000,1000000,10000000]
    pcd = [[0] * len(fake) for i in range(len(thd))]
    alist=[]
    for i in np.arange(len(fake)):
        if thd[j]<=fake[i]<thd[j+1]:
            pcd[j].append(fake[i])
        else:
            print(j)
            print(fake[i])
            j=j+1
            pcd[j].append(fake[i])
    for j in np.arange(len(thd)):
        alist.append(sum(pcd[j])/SUM*100)
    return alist
def add_labels(rects):
    for rect in rects:
        height = rect.get_height()
        plt.text(rect.get_x() + rect.get_width() / 2, height, height, ha='center', va='bottom')
        rect.set_edgecolor('white')
##Local vs Global
#fname="Local vs Global"
def plt_figure1(merge,tool):
    #fname="Based-Cluster vs Global"
    bins=50
    alpha=1
    localcolor='darkgray'
    globalcolor='dimgray'
    fig1 = plt.figure(num=1, figsize=(12, 12), dpi=600)
    plt.subplots_adjust( wspace=.1, hspace=.1 )
    if 2:
        ax1=plt.subplot(3,1,2)
        medianprops = dict(linestyle='-', linewidth=3, color='black')
        sns.violinplot(data=merge,width=0.9,inner=None,color="w", scale='width')
        bplot=boxplot(merge,showfliers=False,positions=[0,1,2,3],patch_artist = True,widths=0.10,medianprops =medianprops)

       # ax1.set_title('Performance comparison',fontsize=16)
        ax1.set_xlabel("cluster purity and completeness distribution",fontsize=14)  
        colors = [localcolor, globalcolor, localcolor, globalcolor,localcolor, globalcolor]#颜色填充
        for patch, color in zip(bplot['boxes'], colors):patch.set_facecolor(color)      
        ax1.tick_params(labelsize=12)
        plt.setp(ax1, xticks=[0.5,2.5,4.5],xticklabels=['completeness','purity','       '])
        ax1.legend([bplot["boxes"][0], bplot["boxes"][1]], [ 'Local Clustering','Based-Cluster '+tool],bbox_to_anchor=(0.5, 1.0),fontsize=14)   #图例 
    plt.tight_layout();
    #fig1.savefig(fname + '.pdf', format='pdf') 
    plt.show()
def plt_figure2(merge,df_local_purity,df_global_purity,tool):
    fname="Based-Cluster vs Global"
    bins=50
    alpha=1
    localcolor='darkgray'
    globalcolor='dimgray'
    fig1 = plt.figure(num=1, figsize=(12, 12), dpi=600)
    plt.subplots_adjust( wspace=.1, hspace=.1 )
    if 1:
        ax2=plt.subplot(3,1,1)
       # ax2.set_title('Cluster size comparison',fontsize=16)
        ax2.set_ylabel("% of clusters",fontsize=14) 
        ax2.set_xlabel("cluster size(log10)",fontsize=14)  
        values=percent(df_local_purity['c_count'].values)
        local= ax2.bar(np.arange(len(values)),values,color=localcolor,alpha=alpha,label='Based-Cluster '+tool) 

        values=percent(df_global_purity['c_count'].values)
        Global= ax2.bar(np.arange(len(values)),values,color=globalcolor,alpha=alpha,label='Global')
        plt.legend(loc='best',fontsize=14)
        scale_ls = np.arange(len(values))
        index_ls = [' ',' ',' ',
                   '2',' ',' ',' ',' ',' ',' ',' ',' ',' ',
                   '3',' ',' ',' ',' ',' ',' ',' ',' ',' ',
                   '4',' ',' ',' ',' ',' ',' ',' ',' ',' ',
                   '5',' ',' ',' ',' ',' ',' ',' ',' ',' ',
                   '6',' ',' ',' ',' ',' ',' ',' ',' ',' ',
                   '7']
        plt.xticks(scale_ls,index_ls)  ## 可以设置坐标字
    plt.tight_layout();
    fig1.savefig(fname + '.pdf', format='pdf') 
    plt.show()

In [26]:
dfGlobal=new_species_df[['readname','clark_clu_species']]
dfGlobal.columns=['node','cluster']
ret,dfGlobal=function.analysize(label,dfGlobal)
print(dfGlobal.head())
print(dfGlobal.shape)

{'Total #reads': 82279710, 'Total #clusters': 3436, 'ground #truth': 549, 'Filtered #clusters': 3436, 'Filtered #reads': 82279710, 'Filtered #ground_truth': 549, 'mean cluster size:': 23946.36495925495, 'median cluster size:': 6.0, '%100 purification:': 0.43324412726732087, '>%95 purification:': 0.4739815640796908, '>%90 purification:': 0.4983645554564377, 'mean purification:': 0.7814664069866769, 'median purification:': 0.8936170212765957, '>%80 purification:': 0.5566458519179304, '>%70 purification:': 0.6193874516800476, '>%60 purification:': 0.7374368123699078, '>%50 purification:': 0.8620279512340172, '>%40 purification:': 0.9179304192685103, '>%30 purification:': 0.9717514124293786, '>%20 purification:': 0.9970264644662503, '>%10 purification:': 1.0, '>%5  purification:': 1.0, '%100 completeness:': 0.00546448087431694, '>%90 completeness:': 0.424408014571949, '>%80 completeness:': 0.6612021857923497, 'mean completeness:': 0.8199743867979359, 'median completeness:': 0.8771234428086

In [27]:
df_local_purity=dfLocal.groupby(['cid'])[['c_count','purification']].max().reset_index().sort_values('c_count',ascending=False)
df_local_completeness=dfLocal.groupby(['rname'])[['r_count','completeness']].max().reset_index().sort_values('r_count',ascending=False)
df_global_purity=dfGlobal.groupby(['cid'])[['c_count','purification']].max().reset_index().sort_values('c_count',ascending=False)
df_global_completeness=dfGlobal.groupby(['rname'])[['r_count','completeness']].max().reset_index().sort_values('r_count',ascending=False)
merge=[df_local_completeness['completeness']*100,df_global_completeness['completeness']*100,\
      df_local_purity['purification']*100,df_global_purity['purification']*100]

In [39]:
plt_figure1(merge,"Clark")


IndexError: list index out of range